In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import sys
sys.path.append('../Utilities/')

from tqdm.notebook import tqdm

import importlib
import data_utils
importlib.reload(data_utils)

## Import MDS from sklearn
from sklearn.manifold import MDS

mds = MDS(n_components=1, random_state=0, normalized_stress='auto')

In [70]:
class UNet1D(nn.Module):
    def __init__(self, in_channels, out_channels, depth=2, num_layers=2):
        super(UNet1D, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_layers = num_layers
        self.depth = depth
        self.encoder = nn.ModuleList()
        self.decoder = nn.ModuleList()
        self.num_start_filters = 32

        self._create_unet(self.in_channels, self.num_start_filters)
        self.bottleneck = nn.Sequential(
            nn.Conv1d(self.num_start_filters * 2 ** (self.depth - 1), 2 * self.num_start_filters * 2 ** (self.depth - 1), kernel_size=1, padding=0),
            nn.ReLU()
        )
        self.logits = nn.Conv1d(self.num_start_filters, self.out_channels, 1, 1)


    def _create_encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU()
        )

    def _create_decoder_block(self, in_channels, out_channels):
        return nn.ModuleList([nn.ConvTranspose1d(in_channels, in_channels//2, kernel_size=2, stride=2),
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU()])

    def _create_unet(self, in_channels, out_channels):
        for _ in range(self.depth):
            self.encoder.append(self._create_encoder_block(in_channels, out_channels))
            in_channels, out_channels = out_channels, out_channels*2

        out_channels = in_channels
        in_channels = in_channels * 2
        for _ in range(self.depth):
            self.decoder.append(self._create_decoder_block(in_channels, out_channels))
            in_channels, out_channels = out_channels, out_channels//2

    def forward(self, x):
        encoded = []
        for enc in self.encoder:
            x = enc(x)
            encoded.append(x)
            x = nn.MaxPool1d(kernel_size=2, stride=2)(x)

        x = self.bottleneck(x)  # Bottleneck layer

        for dec in self.decoder:
            ## Adding input with encoder concatenation
            enc_output = encoded.pop()
            x = dec[0](x)
            x = torch.cat((enc_output, x), dim=1)
            x = dec[1](x)
            x = dec[2](x)
        return self.logits(x)

input_channels = 6 
output_channels = 1 
depth = 4
num_layers = 2

model = UNet1D(input_channels, output_channels, depth, num_layers)
print(model)

UNet1D(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Conv1d(6, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
    )
    (3): Sequential(
      (0): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
    )
  )
  (decoder): ModuleList(
    (0): ModuleList(
      (0): ConvTranspose1d(512, 256, kernel_size=(2,), stride=(2,))
      (1): Conv1d(512, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): ReLU()
    )
    (1): ModuleList(
      (0): ConvTranspose1d(256, 128, kernel_size=(2,), stride=(2,))
      (1): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): ReLU()
    )
    (2): ModuleList(
      (0): ConvTranspose1d(128, 64, kernel_size=(2,), stride=(2,))
      (1): C

In [71]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [72]:
print("Parameters: ",count_parameters(model))

Parameters:  1133057


In [73]:
# Generate synthetic data
num_superpixels = 300
num_features = 6
synthetic_data = np.random.rand(num_superpixels, num_features)
synthetic_data = torch.tensor(synthetic_data, dtype=torch.float32)

#Reshape
synthetic_data = synthetic_data.unsqueeze(0).transpose(1, 2)

# Pass the synthetic data through the U-Net model
with torch.no_grad():
    output = model(synthetic_data)

print("Input shape:", synthetic_data.shape)
print("Output shape:", output.shape)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 37 but got size 40 for tensor number 1 in the list.

In [6]:
class CloudSegmentationModel(nn.Module):
    def __init__(self):
        super(CloudSegmentationModel, self).__init__()
        self.unet = UNet1D(in_channels=6, out_channels=1)
        
    def forward(self, x):
        return torch.sigmoid(self.unet(x))

In [7]:
model = CloudSegmentationModel()

with torch.no_grad():
    output = model(synthetic_data)

print("Input shape:", synthetic_data.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([1, 6, 300])
Output shape: torch.Size([1, 1, 300])


### Creating Dataset

In [8]:
patches,mask = data_utils.get_patch(path_to_folders_images = "../Dataset/Natural_False_Color/", path_to_folders_labels = "../Dataset/Entire_scene_gts/")

c:\Users\GP65\anaconda3\lib\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [38]:
X = []
y = []
for i,j in tqdm(list(zip(patches,mask))):
    try:
        output = data_utils.convert_image_array_to_slic_with_properties(i,j,n_segments=300) 

        ## Getting the X and y arrays
        X_array = np.array([list(list(i.values())[0]) + list(i.values())[1:] for i in output[1]])
        y_array = np.array(output[2])

        ## Normalizing the X_array columwise
        X_array[:,0] = X_array[:,0]/255
        X_array[:,1] = X_array[:,1]/255
        X_array[:,2] = X_array[:,2]/255
        X_array[:,3] = X_array[:,3]/512
        X_array[:,4] = X_array[:,4]/512
        X_array[:,5] = X_array[:,5]/1000



        ## Pad the X_array with -1 and y_array with 0 upto 300
        X_array = np.pad(X_array,((0,300-X_array.shape[0]),(0,0)),mode='constant',constant_values=-1)
        y_array = np.pad(y_array,(0,300-y_array.shape[0]),mode='constant',constant_values=0)

        ## Ordering
        ordering = mds.fit_transform(X_array[:,3:5]).reshape(-1)
        X_array = X_array[ordering.argsort()]
        y_array = y_array[ordering.argsort()].reshape(-1,1)

        ## Appending
        X.append(X_array)
        y.append(y_array)
        
    except KeyboardInterrupt:
        break
    except:
        pass

  0%|          | 0/2124 [00:00<?, ?it/s]

In [45]:
# np.save('../Dataset/X.npy', np.array(X))
# np.save('../Dataset/Y.npy', np.array(y))

X = np.load('../Dataset/X.npy')
y = np.load('../Dataset/Y.npy')

In [46]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_item = torch.tensor(self.X[idx], dtype=torch.float32).T
        y_item = torch.tensor(self.y[idx], dtype=torch.float32).T
        return x_item, y_item

def create_dataloader(X, y, batch_size=32, shuffle=True):
    dataset = CustomDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [47]:
train_loader = create_dataloader(X[:50],y[:50],batch_size=32,shuffle=True)
test_loader = create_dataloader(X[50:],y[50:],batch_size=32,shuffle=True)

## Training the module

In [48]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
segmentationModel = CloudSegmentationModel().to(device)

# train_loader = None # Train loader for our dataset
# test_loader = None # Test loader for our dataset

# loss function and optimizer
criterion = DiceBCELoss()
optimizer = torch.optim.Adam(segmentationModel.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
segmentationModel.train() 

for epoch in tqdm(range(num_epochs)):
    running_loss = 0

    for superpixel, label in train_loader:
        superpixel = superpixel.to(device)
        label = label.to(device)

        # Forward pass
        output = segmentationModel(superpixel)
        loss = criterion(output, label)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Evaluation
    segmentationModel.eval()
    test_loss = 0

    with torch.no_grad():
        for superpixel, label in test_loader:
            superpixel = superpixel.to(device)
            label = label.to(device)

            output = segmentationModel(superpixel)
            test_loss += criterion(output, label).item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1/1000], Loss: 1.6728
Test Loss: 1.1700
Epoch [2/1000], Loss: 1.6719
Test Loss: 1.1707
Epoch [3/1000], Loss: 1.6705
Test Loss: 1.1710
Epoch [4/1000], Loss: 1.6588
Test Loss: 1.1695
Epoch [5/1000], Loss: 1.6377
Test Loss: 1.1708
Epoch [6/1000], Loss: 1.5870
Test Loss: 1.1761
Epoch [7/1000], Loss: 1.5468
Test Loss: 1.1867
Epoch [8/1000], Loss: 1.5087
Test Loss: 1.1936
Epoch [9/1000], Loss: 1.4922
Test Loss: 1.1962
Epoch [10/1000], Loss: 1.4727
Test Loss: 1.1957
Epoch [11/1000], Loss: 1.4855
Test Loss: 1.1953
Epoch [12/1000], Loss: 1.4811
Test Loss: 1.1951
Epoch [13/1000], Loss: 1.4781
Test Loss: 1.1955
Epoch [14/1000], Loss: 1.4715
Test Loss: 1.1966
Epoch [15/1000], Loss: 1.4747
Test Loss: 1.1957
Epoch [16/1000], Loss: 1.4725
Test Loss: 1.1957
Epoch [17/1000], Loss: 1.4810
Test Loss: 1.1961
Epoch [18/1000], Loss: 1.4878
Test Loss: 1.1958
Epoch [19/1000], Loss: 1.4740
Test Loss: 1.1943
Epoch [20/1000], Loss: 1.4684
Test Loss: 1.1954
Epoch [21/1000], Loss: 1.4943
Test Loss: 1.1951
E

KeyboardInterrupt: 

In [62]:
for i in np.round(segmentationModel(superpixel).cpu().detach().numpy(), 0):
    print(i)

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1.
  0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.
  0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [64]:
for i in label:
    print(i)

tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
         1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [31]:
label.shape

torch.Size([32, 300])